In [13]:
# Standard Libraries
import os
import random
import pydicom
from PIL import Image
import yaml

# Data Manipulation Libraries
import pandas as pd
import numpy as np


# Machine Learning Libraries
import torch
from sklearn.model_selection import train_test_split

# YOLO library
from ultralytics import YOLO

**Define Parameters**

In [14]:
# Disease labels
disease_labels = [
    "Aortic enlargement", "Atelectasis", "Calcification", "Cardiomegaly",
    "Consolidation", "ILD", "Infiltration", "Lung Opacity",
    "Nodule/Mass", "Other lesion", "Pleural effusion", "Pleural thickening",
    "Pneumothorax", "Pulmonary fibrosis"
]

# File path
EXTRACTED_PATH = '/cluster/home/bjorneme/projects/Data/vinbigdata-chest-xray-abnormalities-detection-extracted'

# Define parameters
SEED = 42
NUM_WORKERS =32

# Device Configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


**Set Seed or Reproducibity**

In [15]:
def seed_everything(seed=SEED):
    """
    Sets the seed to ensure reproducibility.
    """
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Apply the seed
seed_everything()

# **Helper Functions**

In [16]:
def open_dicom_image(path):
    ds = pydicom.dcmread(path)
    img = ds.pixel_array.astype(np.float32)
    img = ((img - img.min()) / (img.max() - img.min()) * 255).astype(np.uint8)
    return Image.fromarray(img)

In [17]:
def convert_annotation_to_yolo(group, img_size_override=None):
    img_path = group.iloc[0]['Path']
    img_width, img_height = open_dicom_image(img_path).size
    
    lines = []
    for _, row in group.iterrows():
        x_min, y_min, x_max, y_max = row[['x_min','y_min','x_max','y_max']]
        x_center = ((x_min + x_max) / 2) / img_width
        y_center = ((y_min + y_max) / 2) / img_height
        box_width = (x_max - x_min) / img_width
        box_height = (y_max - y_min) / img_height
        lines.append(f"{int(row['class_id'])} {x_center:.6f} {y_center:.6f} {box_width:.6f} {box_height:.6f}")
    return img_path, lines

In [18]:
def process_split(df, split_name, base_dir):
    for image_id, group in df.groupby('image_id'):
        img_path, yolo_lines = convert_annotation_to_yolo(group)
        img_filename = os.path.basename(img_path)
        ext = os.path.splitext(img_filename)[1].lower()
        img_dir = os.path.join(base_dir, "images", split_name)
        label_dir = os.path.join(base_dir, "labels", split_name)
        os.makedirs(img_dir, exist_ok=True)
        os.makedirs(label_dir, exist_ok=True)
        
        pil_img = open_dicom_image(img_path)
        img_filename = os.path.splitext(img_filename)[0] + ".png"
        pil_img.save(os.path.join(img_dir, img_filename))
        
        label_filename = os.path.splitext(img_filename)[0] + ".txt"
        with open(os.path.join(label_dir, label_filename), "w") as f:
            f.write("\n".join(yolo_lines))

# **Step 1: Data Preprocessing**

In [19]:
def load_labels(csv_path, image_path):
    """
    Loads and preprocesses the labels from the CSV file.
    Maps each image to its corresponding file path and binary labels for each disease.
    """

    # Read the CSV file containing labels
    labels_df = pd.read_csv(csv_path)

    # Create binary columns for each disease label
    for disease in disease_labels:
        labels_df[disease] = labels_df['class_name'].str.contains(disease).astype(int)
    
    # Create a binary column for 'No Finding'
    labels_df['No finding'] = labels_df['class_name'].apply(lambda x: 1 if 'No finding' in x else 0)

    # Add the full image path
    labels_df['Path'] = labels_df['image_id'].apply(
        lambda x: os.path.join(image_path, 'train', f"{x}.dicom")
    )

    return labels_df

# Path to the labels CSV file
labels_csv_path = os.path.join(EXTRACTED_PATH, 'train.csv')

# Load and preprocess the labels
labels_df = load_labels(labels_csv_path, EXTRACTED_PATH)

**Filter Images with Finding**

In [20]:
labels_df = labels_df[labels_df['class_id'] != 14]

**Split Dataset**

In [21]:
# Split based on image id
unique_ids = labels_df['image_id'].unique()

# Split patients into training and validation
train_ids, val_ids = train_test_split(
    unique_ids, test_size=0.2, random_state=SEED
)

# Create dataframes for training and validation sets
train_df = labels_df[labels_df['image_id'].isin(train_ids)]
val_df = labels_df[labels_df['image_id'].isin(train_ids)]

# Verify Split Sizes
print(f"Train size: {len(train_ids)}")
print(f"Val size: {len(val_ids)}")

Train size: 3515
Val size: 879


# **Step 2: Load Data**

In [22]:
for split in ['train', 'val']:
    os.makedirs(os.path.join(EXTRACTED_PATH, "images", split), exist_ok=True)
    os.makedirs(os.path.join(EXTRACTED_PATH, "labels", split), exist_ok=True)

print("Processing training set...")
# process_split(train_df, "train", EXTRACTED_PATH)
print("Processing validation set...")
# process_split(val_df, "val", EXTRACTED_PATH)

data_yaml = {
    "train": os.path.join(os.getcwd(), EXTRACTED_PATH, "images", "train"),
    "val": os.path.join(os.getcwd(), EXTRACTED_PATH, "images", "val"),
    "nc": 14,
    "names": disease_labels
}

with open("data.yaml", "w") as f:
    yaml.dump(data_yaml, f, default_flow_style=False)

Processing training set...
Processing validation set...


# **Step 3: Build the Model**

In [23]:
model = YOLO("yolo12s.pt")
model.to(device)

YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C3k2(
        (cv1): Conv(
          (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(96, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_runnin

# **Step 4: Train the Model**

In [24]:
# Train with image size 640
results = model.train(data="data.yaml", epochs=100, imgsz=640, workers=32)

New https://pypi.org/project/ultralytics/8.3.94 available 😃 Update with 'pip install -U ultralytics'
engine/trainer: task=detect, mode=train, model=yolo12s.pt, data=data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cuda:0, workers=32, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, sho

train: Scanning /cluster/home/bjorneme/projects/Data/vinbigdata-chest-xray-abnormalities-detection-extracted/labels/train.cache... 3515 images, 0 backgrounds, 0 corrupt: 100%|██████████| 3515/3515 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Scanning /cluster/home/bjorneme/projects/Data/vinbigdata-chest-xray-abnormalities-detection-extracted/labels/val.cache... 4394 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4394/4394 [00:00<?, ?it/s]
/cluster/home/bjorneme/.local/lib/python3.11/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 64 worker processes in total. Our suggested max number of worker in current system is 32, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Plotting labels to runs/detect/train2/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000556, momentum=0.9) with parameter groups 113 weight(decay=0.0), 120 weight(decay=0.0005), 119 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 32 dataloader workers
Logging results to runs/detect/train2
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all       4394      36096      0.626      0.489       0.54      0.332

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


    100/100      6.21G      1.263      1.108       1.34         99        640: 100%|██████████| 220/220 [00:17<00:00, 12.70it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 138/138 [00:13<00:00, 10.07it/s]


                   all       4394      36096      0.634      0.491      0.545      0.337

100 epochs completed in 1.087 hours.
Optimizer stripped from runs/detect/train2/weights/last.pt, 18.9MB
Optimizer stripped from runs/detect/train2/weights/best.pt, 18.9MB

Validating runs/detect/train2/weights/best.pt...
Ultralytics 8.3.80 🚀 Python-3.11.8 torch-2.4.1+cu121 CUDA:0 (NVIDIA H100 80GB HBM3, 81117MiB)
YOLOv12s summary (fused): 159 layers, 9,236,298 parameters, 0 gradients, 21.2 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 138/138 [00:16<00:00,  8.36it/s]


                   all       4394      36096      0.635       0.49      0.545      0.337
    Aortic enlargement       3067       7162      0.807      0.566        0.7      0.475
           Atelectasis        186        279      0.517      0.566      0.535      0.326
         Calcification        452        960      0.472      0.377      0.406      0.218
          Cardiomegaly       2300       5427      0.873      0.602      0.735       0.54
         Consolidation        353        556      0.613      0.489      0.541      0.354
                   ILD        386       1000      0.572      0.603      0.612      0.404
          Infiltration        613       1247      0.642        0.5      0.567      0.344
          Lung Opacity       1322       2483        0.6      0.512      0.545      0.329
           Nodule/Mass        826       2580      0.636      0.365      0.419      0.243
          Other lesion       1134       2203      0.577      0.424      0.471      0.273
      Pleural effusio